# Programming Assignment 3
## Arnav Thaker
### Math 452, SP22

---

### Problem 1

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

class model(nn.Module):
    def __init__(self, inputSize, hiddenSize, numClasses):
        super().__init__()
        self.fc1 = nn.Linear(inputSize, hiddenSize)
        self.fc2 = nn.Linear(hiddenSize, numClasses)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    def reshape(images, size1, size2):
        return images.reshape(size1, size2)

# model vars
inputSize = 784
hiddenSize = 500
numClasses = 10

# training vars
numEpochs = 5
minBatch = 10
lr = 0.1

# accuracy vars
trainCorrect = 0
trainTotal = 0
testCorrect = 0
testTotal = 0

# create model
myModel = model(inputSize, hiddenSize, numClasses)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(myModel.parameters(), lr = lr)

# load data
MNISTTransform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
trainSet = torchvision.datasets.MNIST(root = './data', train = True, download = False, transform = MNISTTransform)
trainLoader = torch.utils.data.DataLoader(trainSet, batch_size = minBatch, shuffle = True)

testSet = torchvision.datasets.MNIST(root = './data', train = False, download = False, transform = MNISTTransform)
testLoader = torch.utils.data.DataLoader(testSet, batch_size = 1, shuffle = False)

# train
for epoch in range(numEpochs):
    for i, (images, labels) in enumerate(trainLoader):
        images = model.reshape(images, images.size(0), 1*28*28)
        
        outputs = myModel(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs, 1)
        trainTotal += labels.size(0)
        trainCorrect += (predicted == labels).sum().item()

    for i, (images, labels) in enumerate(testLoader):
        images = model.reshape(images, images.size(0), 1*28*28)
        outputs = myModel(images)
        _, predicted = torch.max(outputs, 1)
        testTotal += labels.size(0)
        testCorrect += (predicted == labels).sum().item()
   
    print('Epoch: {}, Training Accuracy: {}%, Test Accuracy: {}%' 
          .format(epoch + 1, (trainCorrect / trainTotal) * 100, (testCorrect / testTotal) * 100))





Epoch: 1, Training Accuracy: 93.33%, Test Accuracy: 96.53%
Epoch: 2, Training Accuracy: 95.29416666666667%, Test Accuracy: 96.815%
Epoch: 3, Training Accuracy: 96.25555555555556%, Test Accuracy: 97.15666666666667%
Epoch: 4, Training Accuracy: 96.86291666666666%, Test Accuracy: 97.43%
Epoch: 5, Training Accuracy: 97.31666666666666%, Test Accuracy: 97.58%


---

### Problem 2

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision



class dnnModel(nn.Module):
    def __init__(self, inputSize, hiddenSize, numClasses):
        super().__init__()
        self.fc1 = nn.Linear(inputSize, inputSize) # hidden layer 1
        self.fc2 = nn.Linear(inputSize, hiddenSize) # hidden layer 2
        self.fc3 = nn.Linear(hiddenSize, numClasses) # output layer
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def logisticRegressionModel(inputSize, numClasses):
    return nn.Linear(inputSize, numClasses)

# model vars
inputSize = 3072
hiddenSize = 500
numClasses = 10

# training vars
numEpochs = 10
minBatch = 100
lr = 0.1

# accuracy vars
trainCorrect = 0
trainTotal = 0
testCorrect = 0
testTotal = 0

# create models
    # LR
lrModel = logisticRegressionModel(inputSize, numClasses)
lrCriterion = nn.CrossEntropyLoss()
lrOptimizer = optim.SGD(lrModel.parameters(), lr = lr)

    # DNN
myDnnModel = dnnModel(inputSize, hiddenSize, numClasses)
dnnCriterion = nn.CrossEntropyLoss()
dnnOptimizer = optim.SGD(myDnnModel.parameters(), lr = lr)

# load cifar data
CIFAR10_transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

cifarTrainset = torchvision.datasets.CIFAR10(root='./data', train=True, download= False, transform=CIFAR10_transform)
cifarTrainLoader = torch.utils.data.DataLoader(cifarTrainset, batch_size=minBatch, shuffle=True)

cifarTestset = torchvision.datasets.CIFAR10(root='./data', train=False, download= False, transform=CIFAR10_transform)
cifarTestLoader = torch.utils.data.DataLoader(cifarTestset, batch_size=1, shuffle=False)

# train
    # LR model
for epoch in range(numEpochs):
    for i, (images, labels) in enumerate(cifarTrainLoader):
        images = images.reshape(images.size(0), 3*32*32)
        outputs = lrModel(images)
        loss = lrCriterion(outputs, labels)
        print(images.size())
        print(outputs.size())
        
        lrOptimizer.zero_grad()
        loss.backward()
        lrOptimizer.step()
        
        _, predicted = torch.max(outputs, 1)
        trainTotal += labels.size(0)
        trainCorrect += (predicted == labels).sum().item()
        
    for i, (images, labels) in enumerate(cifarTestLoader):
        images = images.reshape(images.size(0), 3*32*32)
        outputs = lrModel(images)
        _, predicted = torch.max(outputs, 1)
        testTotal += labels.size(0)
        testCorrect += (predicted == labels).sum().item()

    
    print('Logistic Regression, Epoch: {}, Training Accuracy: {}%, Test Accuracy: {}%' 
          .format(epoch + 1, (trainCorrect / trainTotal) * 100, (testCorrect / testTotal) * 100))

    # reset vars
trainCorrect = 0
trainTotal = 0
testCorrect = 0
testTotal = 0
numEpochs = 30

    # DNN model
for epoch in range(numEpochs):
    for i, (images, labels) in enumerate(cifarTrainLoader):
        images = images.reshape(images.size(0), 3*32*32)
        dnnOutputs = myDnnModel(images)
        dnnLoss = dnnCriterion(dnnOutputs, labels)
        
        dnnOptimizer.zero_grad()
        dnnLoss.backward()
        dnnOptimizer.step()
        
        _, dnnPredicted = torch.max(dnnOutputs, 1)
        trainTotal += labels.size(0)
        trainCorrect += (dnnPredicted == labels).sum().item()

    for i, (images, labels) in enumerate(cifarTestLoader):
        images = images.reshape(images.size(0), 3*32*32)
        dnnOutputs = myDnnModel(images)
        _, dnnPredicted = torch.max(dnnOutputs, 1)
        testTotal += labels.size(0)
        testCorrect += (dnnPredicted == labels).sum().item()
   
    print('DNN Model, Epoch: {}, Training Accuracy: {}%, Test Accuracy: {}%' 
          .format(epoch + 1, (trainCorrect / trainTotal) * 100, (testCorrect / testTotal) * 100))



torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size([100, 10])
torch.Size([100, 3072])
torch.Size

KeyboardInterrupt: 